In [23]:
#Importing required libraries
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications import ResNet50V2
import numpy as np
import sys

In [24]:
#configurations for the model
IMG_SIZE = 224
NUM_CLASSES = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 1
DATA_SAMPLE_SIZE = 5000

Data Loading & Preprocessing

In [25]:
def preprocess_data(image, label):
  """Resizes & normalizes image data for the ResNet-50 model."""
  #converting image & label to float
  image = tf.cast(image, np.float32)

  #resizing the 32x32 CIFAR-10 images to 224x224 size for resnet
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))

  #normalizing the image
  image = image/255.0

  #one-hot encoding the label
  label = tf.one_hot(label, NUM_CLASSES)

  return image, label

In [26]:
def load_cifar10():
  """Loads & prepares CIFAR-10 dataset using TensorFlow Datasets API."""
  #load CIFAR-10 data
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

  #Subsampling the data to reduce training time
  #Using 5000 for train and 1000 for test
  x_train = x_train[:DATA_SAMPLE_SIZE]
  y_train = y_train[:DATA_SAMPLE_SIZE]

  test_sample_size = DATA_SAMPLE_SIZE // 5
  x_test = x_test[:test_sample_size]
  y_test = y_test[:test_sample_size]

  #converting labels to 1D array
  y_train = np.squeeze(y_train)
  y_test = np.squeeze(y_test)

  #create tensorflow datasets
  train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

  #applying preprocessing shuffling & batching
  AUTOTUNE = tf.data.AUTOTUNE

  #training set pipeline
  train_ds = train_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
  train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

  #training set pipeline
  test_ds = test_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
  test_ds = test_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

  return train_ds, test_ds

Model Defintion (Sequential API)

In [27]:
def build_transfer_model():
  """Builds a Keras Functional model using ResNet-50 feature extractor from TF Hub."""

  #Define Input Tensor
  inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

  #resnet50 base model
  base_model = ResNet50V2(
      include_top=False,
      weights='imagenet',
      input_tensor=inputs,
      pooling='avg'
  )

  #freeze the weights of the resnet layers
  base_model.trainable = False

  #output of the base model
  x = base_model.output

  #New Classification Head for 10 classes - Stack the new layers on x
  x = tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(
    NUM_CLASSES,
    activation='softmax',
    name='classification_head'
  )(x)

  #Creating the Model instance using the Functional API
  model = tf.keras.Model(inputs=inputs, outputs=outputs, name='resnet50_cifar10_transfer')

  #compile the model
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=['accuracy']
  )

  #printing summary
  model.summary(print_fn=lambda x: print(x, file=sys.stderr, flush=True))

  return model

Execution

In [28]:
#loading the data
train_ds, test_ds = load_cifar10()

In [29]:
#building the model
model = build_transfer_model()

Model: "resnet50_cifar10_transfer"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114, 

In [30]:
#training the model
history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=test_ds
)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1014s 13s/step - accuracy: 0.3451 - loss: 2.0678 - val_accuracy: 0.7630 - val_loss: 0.6783


In [31]:
#evaluating on the test set
loss, accuracy = model.evaluate(test_ds, verbose=0)

In [32]:
print(loss)

0.678324282169342


In [33]:
print(accuracy)

0.7630000114440918
